In [8]:
import os
import logging
import time
from google.cloud import storage
import vertexai
from vertexai.generative_models import GenerativeModel, Part

In [ ]:
import google.auth
from google.cloud import storage
# Check if credentials are correctly set
credentials, project = google.auth.default()
print(f"Project ID: {project}")
print(f"Credentials: {credentials}")

In [5]:
# Configure logging
logging.basicConfig(level=logging.INFO)

# Initialize Vertex AI and Google Cloud Storage
project_id = "flowing-blade-430704-q3"
vertexai.init(project=project_id, location="us-central1")

storage_client = storage.Client()
bucket_name = "gta_images_bucket_01"
bucket = storage_client.bucket(bucket_name)

model = GenerativeModel("gemini-1.5-flash-001")

In [6]:
# Paths
image_dir = "leftImg8bit"
caption_dir = "google_captions"

# Ensure the caption directory exists
os.makedirs(caption_dir, exist_ok=True)

def upload_to_gcs(local_path, gcs_path):
    try:
        blob = bucket.blob(gcs_path)
        blob.upload_from_filename(local_path)
        logging.info(f"Uploaded {local_path} to gs://{bucket_name}/{gcs_path}")
        return f"gs://{bucket_name}/{gcs_path}"
    except Exception as e:
        logging.error(f"Failed to upload {local_path} to GCS: {e}")
        return None

In [7]:
def generate_caption(image_gcs_uri):
    try:
        response = model.generate_content(
            [
                Part.from_uri(image_gcs_uri, mime_type="image/jpeg"),
                "Describe the image in 2-3 sentences.",
            ]
        )
        logging.info(f"Response: {response}")
        return response.text
    except Exception as e:
        logging.error(f"Failed to generate caption for {image_gcs_uri}: {e}")
        return f"Error generating caption: {e}"

In [8]:
# Add a delay between requests to avoid hitting the quota limit
def rate_limited_generate_caption(image_gcs_uri):
    try:
        # Wait for 30 seconds before making a new request
        time.sleep(30)
        return generate_caption(image_gcs_uri)
    except Exception as e:
        logging.error(f"Failed to generate caption for {image_gcs_uri}: {e}")
        return f"Error generating caption: {e}"

# Initialize a counter for processed images
processed_count = 0

In [9]:
# Traverse the directory structure
for split in ['test', 'train', 'val']:
    split_dir = os.path.join(image_dir, split)
    for city in os.listdir(split_dir):
        city_dir = os.path.join(split_dir, city)
        
        if not os.path.isdir(city_dir):
            continue  # Skip non-directory files
        
        # List all images in the city folder
        image_files = [f for f in os.listdir(city_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Calculate top 10% of images
        top_10_percent_count = max(1, len(image_files) // 10)
        top_images = sorted(image_files)[:top_10_percent_count]
        
        for file in top_images:
            #if processed_count >= 500:
                #logging.info("Max images processed, exiting...")
                #break

            image_path = os.path.join(city_dir, file)
            caption_file_name = f"{os.path.splitext(file)[0]}.txt"
            caption_subdir = os.path.join(caption_dir, split, city)
            os.makedirs(caption_subdir, exist_ok=True)
            caption_path = os.path.join(caption_subdir, caption_file_name)
            
            # Upload image to GCS
            gcs_path = f"temp_images/{file}"
            image_gcs_uri = upload_to_gcs(image_path, gcs_path)
            if not image_gcs_uri:
                continue

            # Generate caption with rate limiting
            caption = rate_limited_generate_caption(image_gcs_uri)

            # Save caption to file
            try:
                with open(caption_path, 'w') as caption_file:
                    caption_file.write(caption)
                logging.info(f"Saved caption to {caption_path}")
            except Exception as e:
                logging.error(f"Failed to save caption to {caption_path}: {e}")
            
            # Optionally, delete the uploaded image from GCS
            try:
                bucket.blob(gcs_path).delete()
                logging.info(f"Deleted {gcs_path} from GCS")
            except Exception as e:
                logging.error(f"Failed to delete {gcs_path} from GCS: {e}")

            # Increment the counter
            processed_count += 1

        if processed_count >= 500:
            break
    if processed_count >= 500:
        break

print("Captions generated and saved successfully.")

INFO:root:Uploaded leftImg8bit/test/bielefeld/bielefeld_000000_000321_leftImg8bit.png to gs://gta_images_bucket_01/temp_images/bielefeld_000000_000321_leftImg8bit.png
I0000 00:00:1722390076.528581   11117 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
INFO:root:Response: candidates {
  content {
    role: "model"
    parts {
      text: "The image shows a view from the driver\'s seat of a car, traveling down a street lined with trees and hedges. There are several cars in the distance, and the driver\'s side of the car shows a Mercedes-Benz emblem. The road is paved and appears to be curving slightly. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.027637721970677376
    severity: HARM_SEVERITY_NEGLIGIBLE
 

Captions generated and saved successfully.
